# Bon Poison - Livraisons à domicile

Ce fichier contient plusieurs étapes
1. import de librairies essentielles
2. Insertion du fichier csv de clients pour livraison et geocoding
3. Traitement du fichier et des doublons
4. Clustering

In [5]:
import pylab
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim, GoogleV3, IGNFrance, BANFrance
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium import plugins, IFrame
from folium.features import *

import random
import csv
import simplekml

from pyroutelib3 import Router
import numpy as np

from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist
from scipy.sparse.csgraph import shortest_path

from tsp_solver.greedy import solve_tsp

from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn import mixture

pylab.rcParams['figure.figsize'] = (24.0, 12.0) #configure the figure

def lon_lat_to_xy(lon, lat, R = 6371000.0):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius R
    """
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    x =  R * np.cos(lat_r[0]) * lon_r
    y = R * lat_r
    return x,y

def get_distance(routeLatLons):
    
    data = []
    for iter, point in enumerate(routeLatLons):
        data.append([point[0], point[1]])
    
    columns = ['Latitude', 'Longitude']
    temp = pd.DataFrame(data, columns=columns)
    
    x_temp,y_temp = lon_lat_to_xy(temp['Longitude'], temp['Latitude'], R = 6371000.0)
    x = x_temp - x_temp[0]
    y = y_temp - y_temp[0]
    d_x = np.diff(x)
    d_y = np.diff(y)
    ds = np.sqrt(d_x*d_x + d_y*d_y) 
    return np.sum(ds)

In [6]:
#Lecture du fichier
df = pd.read_csv("17_04/Livraison_17_04.csv", sep=';')
df.astype({'CP': 'int'}).dtypes
#data = []
# always inserting new rows at the first position - last row will be always on top    
#data.insert(0, {'NOM': 'PISCICULTURE', 'PRENOM': 'ROMAN', 'ADRESSE': '36 Chemin du Lavoir', 'CP': 38270, 'VILLE': 'Beaufort'})
#df = pd.concat([pd.DataFrame(data), df], ignore_index=True)

#On affiche l'adresse totale
df['addresse_tot'] = df['ADRESSE'].astype(str) + ',' + \
                df['CP'].astype(str) + ',' + \
                df['VILLE'].astype(str) + ', France'   

#En utilisant un $locator de geopy, on trouve l'adresse et la position
locator = BANFrance()
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)
df['location'] = df['addresse_tot'].apply(geocode)
#On rajoute un point avec des valeurs nulles
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#On sépare ensuite l'objet point en latitude, longitude et altitude
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [9]:
df.columns

Index(['NOM', 'PRENOM', 'ADRESSE', 'CP', 'VILLE', 'Groupe', 'R_IMPERIAL_STOUT',
       'FIREHOUSE', 'SCOTCH_ALE', 'MASS_HYSTERA', 'A_PALE_ALE', 'PEATED',
       'PORTER', 'BROWN_ALE', 'PALE_ALE', 'IPA', 'ORIGINALE', 'TELEPHONE',
       'COM', 'addresse_tot', 'location', 'point', 'latitude', 'longitude',
       'altitude', 'index', 'addresse_BAN'],
      dtype='object')

In [10]:
#On ajoute l'adresse exacte dans un champ 
df['index'] = np.arange(len(df))
df['addresse_BAN'] = df.location.astype(str)
df['addresse_BAN'] = df['addresse_BAN'].str.split(',', n = 1)
df.addresse_BAN = df.addresse_BAN.str[0]
#On sépare les commandes
df.addresse_BAN[df.addresse_BAN.duplicated()].head()
df['PRENOM'].fillna('', inplace=True)

#dupli = df.duplicated(['addresse_BAN'])
dupli = df[df.duplicated(['NOM', 'addresse_BAN'], keep=False)]

df2 = df.groupby(['NOM','addresse_BAN'], as_index=False)[["R_IMPERIAL_STOUT", "FIREHOUSE", "SCOTCH_ALE", "MASS_HYSTERA", "A_PALE_ALE", "PEATED", "PORTER", "BROWN_ALE", "PALE_ALE", "IPA", "ORIGINALE"]].sum()
df3 = df.drop_duplicates(subset =['NOM','addresse_BAN'], keep ="first", inplace = False)
mergedDf = df2.merge(df3[['index','NOM','PRENOM','ADRESSE','CP','VILLE','addresse_BAN','TELEPHONE',"COM",'latitude', 'longitude','Groupe']], on=['NOM','addresse_BAN'])

mergedDf.sort_values(['index'], ascending=1, inplace=True)
mergedDf.reset_index(drop=True, inplace=True)

print(dupli)

Empty DataFrame
Columns: [NOM, PRENOM, ADRESSE, CP, VILLE, Groupe, R_IMPERIAL_STOUT, FIREHOUSE, SCOTCH_ALE, MASS_HYSTERA, A_PALE_ALE, PEATED, PORTER, BROWN_ALE, PALE_ALE, IPA, ORIGINALE, TELEPHONE, COM, addresse_tot, location, point, latitude, longitude, altitude, index, addresse_BAN]
Index: []

[0 rows x 27 columns]


In [30]:
mergedDf['x'], mergedDf['y'] = lon_lat_to_xy(mergedDf['longitude'], mergedDf['latitude'], R = 6371000.0)
df_array = mergedDf[['x', 'y']].to_numpy()
dist_map = cdist(df_array, df_array)
pd.DataFrame(dist_map)[0].values
to_drop = np.argwhere(pd.DataFrame(dist_map)[0].values > 1.E5)

#df_lat_nulle = mergedDf[mergedDf['latitude'].isnull()]
df_lat_nulle = mergedDf[mergedDf['latitude'].isna()]
for value in to_drop.flatten():
    value_int = int(value)
    df_lat_nulle = df_lat_nulle.append(mergedDf.iloc[value_int]) 
    #mergedDf.drop([value_int])       

df_lat_nulle.drop_duplicates(subset ='addresse_BAN', keep ="first", inplace = True)

final_client = pd.concat([mergedDf, df_lat_nulle, df_lat_nulle]).drop_duplicates(keep=False)
df_lat_nulle.to_csv('clients_erreur.csv', sep=';')
final_client.to_csv('clients_correct.csv', sep=';')

#final_client['commande'] = final_client['Decouverte'].apply(lambda x: str(x) + ' Decouverte ' if x > 0.0 else '')

temp = final_client.copy()[[]]
temp['R_IMPERIAL_STOUT'] = final_client['R_IMPERIAL_STOUT'].apply(lambda x: str(int(x)) + ' R_IMPERIAL_STOUT ' if x > 0.0 else '')
temp['FIREHOUSE'] = final_client['FIREHOUSE'].apply(lambda x: str(int(x)) + ' FIREHOUSE ' if x > 0.0 else '')
temp['SCOTCH_ALE'] = final_client['SCOTCH_ALE'].apply(lambda x: str(int(x)) + ' SCOTCH_ALE Fario ' if x > 0.0 else '')
temp['MASS_HYSTERA'] = final_client['MASS_HYSTERA'].apply(lambda x: str(int(x)) + ' MASS_HYSTERA ' if x > 0.0 else '')
temp['A_PALE_ALE'] = final_client['A_PALE_ALE'].apply(lambda x: str(int(x)) + ' A_PALE_ALE ' if x > 0.0 else '')
temp['PEATED'] = final_client['PEATED'].apply(lambda x: str(int(x)) + ' PEATED ' if x > 0.0 else '')
temp['PORTER'] = final_client['PORTER'].apply(lambda x: str(int(x)) + ' PORTER ' if x > 0.0 else '')
temp['BROWN_ALE'] = final_client['BROWN_ALE'].apply(lambda x: str(int(x)) + ' BROWN_ALE ' if x > 0.0 else '')
temp['PALE_ALE'] = final_client['PALE_ALE'].apply(lambda x: str(int(x)) + ' PALE_ALE ' if x > 0.0 else '')
temp['IPA'] = final_client['IPA'].apply(lambda x: str(int(x)) + ' IPA ' if x > 0.0 else '')
temp['ORIGINALE'] = final_client['ORIGINALE'].apply(lambda x: str(int(x)) + ' ORIGINALE ' if x > 0.0 else '')

final_client['commande'] = temp[['R_IMPERIAL_STOUT', 'FIREHOUSE', 'SCOTCH_ALE', 'MASS_HYSTERA', 'A_PALE_ALE', 'PEATED', 'PORTER', 'BROWN_ALE', 'PALE_ALE', 'IPA', 'ORIGINALE']].apply(lambda x: ''.join(x), axis=1)

In [31]:
df

,NOM,PRENOM,ADRESSE,CP,VILLE,Groupe,R_IMPERIAL_STOUT,FIREHOUSE,SCOTCH_ALE,MASS_HYSTERA,...,TELEPHONE,COM,addresse_tot,location,point,latitude,longitude,altitude,index,addresse_BAN
0,A,toto,18 GALERIE CLEMENT ADER,38200,VIENNE,2,NaN,6.0,NaN,NaN,...,660381219,NaN,"18 GALERIE CLEMENT ADER,38200,VIENNE, France","(18 Galerie Clement Ader 38200 Vienne, (45.513...","(45.513732, 4.911006, 0.0)",45.513732,4.911006,0.0,0,18 Galerie Clement Ader 38200 Vienne
1,B,tata,2685 ROUTE DE LA FORTERESSE,38590,SAINT MICHEL DE SAINT GEOIRS,3,3.0,NaN,3.0,NaN,...,476654234,NaN,"2685 ROUTE DE LA FORTERESSE,38590,SAINT MICHEL...",(2685 Route de la Forteresse 38590 Saint-Miche...,"(45.308331, 5.367319, 0.0)",45.308331,5.367319,0.0,1,2685 Route de la Forteresse 38590 Saint-Michel...
2,C,titi,93 RUE LAFAYETTE,38200,VIENNE,2,NaN,NaN,NaN,12.0,...,677026341,NaN,"93 RUE LAFAYETTE,38200,VIENNE, France","(93 Rue Lafayette 38200 Vienne, (45.527413, 4....","(45.527413, 4.897992, 0.0)",45.527413,4.897992,0.0,2,93 Rue Lafayette 38200 Vienne
3,AA,toto,259 ROUTE DE GRENOBLE,38260,PAJAY,0,NaN,NaN,12.0,NaN,...,786439995,NaN,"259 ROUTE DE GRENOBLE,38260,PAJAY, France","(Route de Grenoble 38260 Pajay, (45.347849, 5....","(45.347849, 5.096872, 0.0)",45.347849,5.096872,0.0,3,Route de Grenoble 38260 Pajay
4,AB,titi,1 RUE DE CHORERIE,42410,CHAVANAY,1,NaN,24.0,NaN,NaN,...,768770221,NaN,"1 RUE DE CHORERIE,42410,CHAVANAY, France","(1 Rue de la Chorerie 42410 Chavanay, (45.4110...","(45.41107, 4.729521, 0.0)",45.411070,4.729521,0.0,4,1 Rue de la Chorerie 42410 Chavanay
5,AC,toto,31 IMPASSE DES CHATAIGNIERS,38260,COMMELLE,2,12.0,NaN,NaN,NaN,...,677741570,NaN,"31 IMPASSE DES CHATAIGNIERS,38260,COMMELLE, Fr...",(31 Impasse des Chataigniers 38260 Porte-des-B...,"(45.438144, 5.237351, 0.0)",45.438144,5.237351,0.0,5,31 Impasse des Chataigniers 38260 Porte-des-Bo...
6,AA,toto,4TER RUE DU PORT VIEUX,38550,LE PEAGE DE ROUSSILLON,1,NaN,NaN,12.0,NaN,...,665258530,NaN,"4TER RUE DU PORT VIEUX,38550,LE PEAGE DE ROUSS...",(4 ter Rue du Port Vieux 38550 Le Péage-de-Rou...,"(45.375975, 4.790213, 0.0)",45.375975,4.790213,0.0,6,4 ter Rue du Port Vieux 38550 Le Péage-de-Rous...
7,AB,toto,10 ROUTE DU PRE GELE,38150,LA CHAPELLE DE SURIEU,1,NaN,NaN,6.0,NaN,...,650549188,NaN,"10 ROUTE DU PRE GELE,38150,LA CHAPELLE DE SURI...",(10 Route du Pre Gele 38150 La Chapelle-de-Sur...,"(45.391941, 4.910289, 0.0)",45.391941,4.910289,0.0,7,10 Route du Pre Gele 38150 La Chapelle-de-Surieu
8,AC,toto,460 RUE DES ARRONDIERES,38850,CHARAVINES,3,NaN,NaN,NaN,NaN,...,630678378,NaN,"460 RUE DES ARRONDIERES,38850,CHARAVINES, France","(460 Rue des Arrondières 38850 Charavines, (45...","(45.428898, 5.522108, 0.0)",45.428898,5.522108,0.0,8,460 Rue des Arrondières 38850 Charavines
9,AA,titi,285 ROUTE DE BOURGOIN JALLIEU,38300,CRACHIER,2,NaN,NaN,NaN,NaN,...,688529063,NaN,"285 ROUTE DE BOURGOIN JALLIEU,38300,CRACHIER, ...","(285 Route de Bourgoin Jallieu 38300 Crachier,...","(45.549077, 5.227175, 0.0)",45.549077,5.227175,0.0,9,285 Route de Bourgoin Jallieu 38300 Crachier


On réalise le clustering

In [32]:
tournees=[]
for region, df_region in final_client.groupby('Groupe'):
    
    #df_region.to_csv('clients_tournee_' + str(region) + '.csv', sep=';')    
    #path = solve_tsp(dist_map_gps, endpoints = (0,-1))
    
    df_array = df_region[['x', 'y']].to_numpy()
    #pisciculture debut et fin
    df_array = np.vstack(((398784.030188979, 5042181.20654134),df_array))
    df_array = np.vstack((df_array, (398784.030188979, 5042181.20654134)))
    dist_map = cdist(df_array, df_array)
    path = solve_tsp(dist_map, endpoints = (0,-1))  
    path_red = path[1:-1]
    sorterIndex = dict(zip(path_red,range(1,len(path_red)+1)))
    df_region_ordered = df_region.copy(deep=True)    
    df_region_ordered['index'] = np.arange(1,len(path_red)+1)
    df_region_ordered['order'] = df_region_ordered['index'].map(sorterIndex)    
    df_region_ordered.sort_values(['order'], ascending=1, inplace=True)
    df_region_ordered.reset_index(drop=True, inplace=True)            
    df_region_reduced = df_region_ordered[["NOM", "PRENOM", "TELEPHONE", "ADRESSE", "CP", "VILLE", "R_IMPERIAL_STOUT", "FIREHOUSE", "SCOTCH_ALE", "MASS_HYSTERA", "A_PALE_ALE", "PEATED", "PORTER", "BROWN_ALE", "PALE_ALE", "IPA", "ORIGINALE"]]    
    df_region_reduced.to_csv('clients_tournee_' + str(int(region)) + '.csv', sep=';')
    tournees.append(df_region_ordered)

In [33]:
data = []
data.insert(0, {'NOM': 'BRASSERIE', 'PRENOM': 'ROMAN', 'ADRESSE': '36 Chemin du Lavoir', 'CP': 38270, 'VILLE': 'Beaufort' \
                , 'latitude': 45.345425, 'longitude': 5.102722, 'x': 398784.030188979, 'y': 5042181.20654134})
final_client = pd.concat([pd.DataFrame(data), final_client], ignore_index=True)

for index, tournee in zip(range(len(tournees)),tournees):
    tournees[index] = pd.concat([pd.DataFrame(data), tournee], ignore_index=False)

In [34]:
tournees[0]


,NOM,PRENOM,ADRESSE,CP,VILLE,latitude,longitude,x,y,addresse_BAN,...,BROWN_ALE,PALE_ALE,IPA,ORIGINALE,index,TELEPHONE,COM,Groupe,commande,order
0,BRASSERIE,ROMAN,36 Chemin du Lavoir,38270,Beaufort,45.345425,5.102722,398784.030189,5.042181e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AA,toto,259 ROUTE DE GRENOBLE,38260,PAJAY,45.347849,5.096872,397140.844750,5.042451e+06,Route de Grenoble 38260 Pajay,...,0.0,0.0,0.0,0.0,1.0,786439995.0,NaN,0.0,12 SCOTCH_ALE Fario,1.0


In [36]:
folium_map = folium.Map(location=[final_client['latitude'].mean(), 
                                final_client['longitude'].mean()], 
                                zoom_start=12, tiles='OpenStreetMap')    

folium.plugins.BoatMarker([final_client.iloc[0]['latitude'], final_client.iloc[0]['longitude']]).add_to(folium_map)

Tournee_0 = FeatureGroup(name='Tournee_0')
Tournee_1 = FeatureGroup(name='Tournee_1')
Tournee_2 = FeatureGroup(name='Tournee_2')
Tournee_3 = FeatureGroup(name='Tournee_3')

for i in range(1,len(final_client)): 
    
    url='https://www.google.fr/maps/dir//' + \
    str(final_client.iloc[i]['latitude'])+ ',' + str(final_client.iloc[i]['longitude'])
#    address = str(final_client.iloc[i]['addresse_BAN'])
    address = str(final_client.iloc[i]['ADRESSE']) + ', ' + str(final_client.iloc[i]['CP']) + ', ' + str(final_client.iloc[i]['VILLE'])    
    nom_prenom = str(final_client.iloc[i]['NOM']) + '\t' + str(final_client.iloc[i]['PRENOM'])
    tel = str(final_client.iloc[i]['TELEPHONE'])
    colis = str(final_client.iloc[i]['commande']) # + ' = ' + str(final_client.iloc[i]['PRIX']) + ' euros'
    lien = '<a href=' + url + '> Maps </a>'
    commentaire = str(final_client.iloc[i]['COM'])
    if commentaire == 'nan':
        commentaire = ''
    
    popup_name = '<b>' + nom_prenom + '</b>' + '<br>' + str(address) + '<br>'  + tel + '<br>' + colis + '<br>' + lien + '<br>' + commentaire
    
#    iframe = folium.IFrame(popup_name, width=330, height=150)
#    iframe = folium.IFrame(html=popup_name)
    popup = folium.Popup(popup_name, max_width=650)
    
    if final_client.iloc[i]['Groupe'] == 0:
        icon = folium.map.Icon(color='blue')
        Tournee_0.add_child(Marker([final_client.iloc[i]['latitude'], final_client.iloc[i]['longitude']], icon=icon, popup=popup))
    if final_client.iloc[i]['Groupe'] == 1:
        icon = folium.map.Icon(color='green')
        Tournee_1.add_child(Marker([final_client.iloc[i]['latitude'], final_client.iloc[i]['longitude']], icon=icon, popup=popup))
    if final_client.iloc[i]['Groupe'] == 2:
        icon = folium.map.Icon(color='red')
        Tournee_2.add_child(Marker([final_client.iloc[i]['latitude'], final_client.iloc[i]['longitude']], icon=icon, popup=popup))
    if final_client.iloc[i]['Groupe'] == 3:
        icon = folium.map.Icon(color='orange')
        Tournee_3.add_child(Marker([final_client.iloc[i]['latitude'], final_client.iloc[i]['longitude']], icon=icon, popup=popup))

#FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values)), popup=df['addresse_tot']).add_to(folium_map)

folium_map.add_child(Tournee_0)
folium_map.add_child(Tournee_1)
folium_map.add_child(Tournee_2)
folium_map.add_child(Tournee_3)
folium.LayerControl().add_to(folium_map)
folium_map

In [26]:
folium_map.save("map_global.html")
folium_map

In [37]:
folium_map = folium.Map(location=[final_client['latitude'].mean(), 
                                    final_client['longitude'].mean()], 
                                    zoom_start=10, tiles='OpenStreetMap')
folium.plugins.BoatMarker([final_client.iloc[0]['latitude'], final_client.iloc[0]['longitude']]).add_to(folium_map)

Tournee_group=[]

for number, tournee in zip(range(len(tournees)),tournees):

    Tournee = FeatureGroup(name='Tournee'+str(int(number)))
    for i in range(1,len(tournee)):

        url='https://www.google.fr/maps/dir//' + \
        str(tournee.iloc[i]['latitude'])+ ',' + str(tournee.iloc[i]['longitude'])
        address = str(tournee.iloc[i]['addresse_BAN'])
        nom_prenom = str(tournee.iloc[i]['NOM']) + '\t' + str(tournee.iloc[i]['PRENOM'])
        tel = str(tournee.iloc[i]['TELEPHONE'])
        colis = str(tournee.iloc[i]['commande']) #+ ' = ' + str(tournee.iloc[i]['PRIX']) + ' euros'
        lien = '<a href=' + url + '> Maps </a>'

        popup_name = '<b>' + nom_prenom + '</b>' + '<br>' + str(address) + '<br>'  + tel + '<br>' + colis + '<br>' + lien

        popup = folium.Popup(popup_name, max_width=650)

        icon_order = DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 14pt">' + str(int(tournee.iloc[i]['order'])) + '</div>',
            )

        if tournee.iloc[i]['Groupe'] == 0:
            icon = folium.map.Icon(color='blue')
        if tournee.iloc[i]['Groupe'] == 1:
            icon = folium.map.Icon(color='green')
        if tournee.iloc[i]['Groupe'] == 2:
            icon = folium.map.Icon(color='red')
        if tournee.iloc[i]['Groupe'] == 3:
            icon = folium.map.Icon(color='orange')
        
        folium.Marker([tournee.iloc[i]['latitude'], tournee.iloc[i]['longitude']], icon=icon_order).add_to(Tournee)
        folium.Marker([tournee.iloc[i]['latitude'], tournee.iloc[i]['longitude']], icon=icon, popup=popup).add_to(Tournee)
        Tournee_group.append(Tournee)
                           
for tournee in Tournee_group:
        folium_map.add_child(tournee)
                           
    #FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values)), popup=df['addresse_tot']).add_to(folium_map)

    #folium_map.add_child(feature_group)
folium.LayerControl().add_to(folium_map)
folium_map.save("map_tournees.html")
folium_map